In [4]:
# REGRESSION ANALYSIS - running of linear regressions on cleaned data

import numpy as np
import pandas as pd 
import statsmodels.api as sm
import statsmodels.formula.api as smf

# all of the census tract data in one big dataframe, created in box-data-4-24 notebook:
%store -r data_matrix

# a list of all the features
print(data_matrix.columns)



Index(['GIS_code', 'census_year', 'st_code', 'total', 'urban', 'rural', 'male',
       'female', 'male15_24', 'hh_income', 'drivers', 'solodrivers',
       'carpoolers', 'pubtrans', 'cyclists', 'walkers', 'under15', 'over45',
       'male16_19', 'inschool_m1619', 'unenrolled_m1619', 'unemployed_m1619',
       'lowest_ratio', 'highest_ratio', 'NumKilled', 'NumInjury', 'NumVeh',
       'NumAcc', 'pop', 'medianAge', 'income', 'pctProverty', 'pctWhite',
       'pctBlack', 'pctOccup', 'pctVacant', 'popQT', 'incomeQT', 'provertyQT',
       'occupQT', 'vacantQT', 'hsQT', 'whiteQT', 'blackQT', 'NumKilled_pctpop',
       'NumInjury_pctpop', 'NumVeh_pctpop', 'NumAcc_pctpop',
       'NumKilled_pctacc'],
      dtype='object')


In [5]:
# a function which uses forward selection to determine the best linear model, based on maximized adj. R^2 
# i didn't find this function very useful because it gave models which included variables with insignificant p-values

def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    remaining = set(data.columns)
    remaining.remove(response)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(selected + [candidate]))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected))
    model = smf.ols(formula, data).fit()
    return model

In [6]:
#for xc in X_all.columns:
    #print(xc)
    #print(np.any(np.isnan(X_all[xc])))
print(data_matrix.shape)   
#print(np.any(np.isnan(data_matrix.male15_24)))

# make sure no 0 values because we'll be dividing by male15_24 
data_matrix = data_matrix[data_matrix.male15_24 != 0]
print(data_matrix.shape)

(1386, 49)
(1385, 49)


In [7]:
# test a model with one variable to see how it looks

mod = smf.ols(formula = 'NumKilled ~ hsQT', data=data_matrix).fit()

print(mod.summary())

                            OLS Regression Results                            
Dep. Variable:              NumKilled   R-squared:                       0.025
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     35.68
Date:                Tue, 02 May 2017   Prob (F-statistic):           2.95e-09
Time:                        23:36:21   Log-Likelihood:                -1800.7
No. Observations:                1385   AIC:                             3605.
Df Residuals:                    1383   BIC:                             3616.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      0.7956      0.055     14.408      0.0

In [8]:
# table of all the accident data:

%store -r accidents

print(accidents.shape)
print(len(accidents.Tract.unique()))
accidents.head()


(109668, 11)
1391


,AccNum,Tract,NumInjury,NumKilled,NumVeh,AccTime,DD,PersonAge,PersonSex,DUI,dayLight
107434,3100031041,29001950100,0,0,2,2100.0,4.0,25.0,2.0,0.0,0.5
86442,3100001595,29001950100,1,0,1,1815.0,2.0,34.0,1.0,1.0,0.0
86453,3100001612,29001950100,1,0,2,1650.0,4.0,55.0,1.0,0.0,1.0
107968,3100031781,29001950100,2,0,1,1800.0,4.0,19.0,2.0,0.0,1.0
103993,3100026242,29001950100,0,0,2,1515.0,4.0,88.0,2.0,0.0,1.0


In [9]:
# Find sums by census tract:

dui_bytract = accidents[['Tract', 'DUI']][accidents.DUI == 1.0].groupby('Tract', as_index=False).count()

total = accidents[['AccNum', 'Tract']].groupby('Tract', as_index=False).count()

#dui_bytract = dui_bytract.assign(totalacc = total.AccNum)

dui_bytract = total.merge(dui_bytract, on='Tract', how='outer')


print(total.shape)
print(dui_bytract.shape)

# the total number of accidents and DUIs in each tract
dui_bytract.loc[np.isnan(dui_bytract.DUI), 'DUI'] = 0



#dui_bytract[np.isnan(dui_bytract.DUI)]
dui_bytract.head()

(1391, 2)
(1391, 3)


,Tract,AccNum,DUI
0,29001950100,31,4.0
1,29001950200,45,7.0
2,29001950300,81,1.0
3,29001950400,32,1.0
4,29001950500,60,3.0


In [10]:
alc_drugs = accidents[['Tract', 'DD']][accidents.DD == 1.0].groupby('Tract', as_index=False).count()
drugs = accidents[['Tract', 'DD']][accidents.DD == 3.0].groupby('Tract', as_index=False).count()
alc = accidents[['Tract', 'DD']][accidents.DD == 2.0].groupby('Tract', as_index=False).count()

alc_drugs = alc_drugs.assign(drugs = drugs.DD, alc = alc.DD)

dui_bytract = dui_bytract.merge(alc_drugs)
dui_bytract.head()

,Tract,AccNum,DUI,DD,alc,drugs
0,29003010100,93,5.0,1,4,1
1,29003010200,78,1.0,1,7,1
2,29003010300,29,1.0,1,1,1
3,29009960100,73,8.0,1,1,3
4,29009960200,81,8.0,1,3,1


In [91]:

# number of DUIs by the raw number of men aged 15-24 per tract 

dui_bytract = dui_bytract.assign(pctDUI_m1524 = np.round(dui_bytract.DUI / data_matrix.male15_24 * 100, 2))

# number of DUIs by the total number of accidents per tract
dui_bytract = dui_bytract.assign(pctDUI = np.round(dui_bytract.DUI / dui_bytract.AccNum * 100, 2))

dui_bytract = dui_bytract.assign(pct_drugs = np.round(alc_drugs.drugs / dui_bytract.AccNum * 100, 2))
dui_bytract = dui_bytract.assign(pct_drugsalc = np.round(alc_drugs.DD / dui_bytract.AccNum * 100, 2))

#for df in intox:
    #print(df.head())
    #dui_bytract = dui_bytract.assign(df.DD)

# add new columns for the normalized y-variables
data_matrix = data_matrix.assign(pctDUI_m1524 = dui_bytract.pctDUI_m1524)
data_matrix = data_matrix.assign(pctDUI = dui_bytract.pctDUI)

data_matrix = data_matrix.assign(pct_drugsalc = dui_bytract.pct_drugsalc)

data_matrix = data_matrix.assign(pct_drugs = dui_bytract.pct_drugs)
data_matrix = data_matrix.assign(male15_24pct = np.round(data_matrix.male15_24 / data_matrix.total * 100, 2))

print(data_matrix.shape)
print(dui_bytract.shape)
print(data_matrix.columns)
#dui_bytract.head()
#data_matrix[['pctDUI', 'NumKilled']].head()
#accidents[['AccNum', 'Tract']].groupby('Tract').count()

(1386, 54)
(217, 10)
Index(['GIS_code', 'census_year', 'st_code', 'total', 'urban', 'rural', 'male',
       'female', 'male15_24', 'hh_income', 'drivers', 'solodrivers',
       'carpoolers', 'pubtrans', 'cyclists', 'walkers', 'under15', 'over45',
       'male16_19', 'inschool_m1619', 'unenrolled_m1619', 'unemployed_m1619',
       'lowest_ratio', 'highest_ratio', 'NumKilled', 'NumInjury', 'NumVeh',
       'NumAcc', 'pop', 'medianAge', 'income', 'pctProverty', 'pctWhite',
       'pctBlack', 'pctOccup', 'pctVacant', 'popQT', 'incomeQT', 'provertyQT',
       'occupQT', 'vacantQT', 'hsQT', 'whiteQT', 'blackQT', 'NumKilled_pctpop',
       'NumInjury_pctpop', 'NumVeh_pctpop', 'NumAcc_pctpop',
       'NumKilled_pctacc', 'male15_24pct', 'pctDUI_m1524', 'pctDUI',
       'pct_drugs', 'pct_drugsalc'],
      dtype='object')


In [10]:
# compare these variables' individual effect:
mod4 = smf.ols(formula = 'NumKilled_pctacc ~ hsQT', data=data_matrix).fit()
mod5 = smf.ols(formula = 'NumKilled_pctacc ~ male', data=data_matrix).fit()
mod6 = smf.ols(formula = 'NumKilled_pctAcc ~ pctBlack', data=data_matrix).fit()
mod7 = smf.ols(formula = 'NumKilled_pctacc ~ pubtrans', data=data_matrix).fit()
mod8 = smf.ols(formula = 'pctDUI ~ carpoolers', data=data_matrix).fit()

mods = [mod4, mod5, mod6]   #, mod6, mod7, mod8]


for m in mods:
    print(m.summary())
# vacancy is a much stronger predictor than poverty
# also a stronger predictor than hh_income

#print(X_trimmed.columns)


# highschool and vacancy are extremely collinear; will keep vacancy (smaller p-val) -- for number of people killed
# by the total population

# number of DUIs / total number of accidents:
# - highschool is a much better predictor than vacancy
# - another notable predictor is % of commuters who carpool to work
# --> percent of commuters who use public transportation is better than carpool though
# - men got off easy on this one... unenrolled/unemployed/male15_24 were are insignificant
# - walkers is insignificant
# - pctWhite is not bad but it has high collinearity with highschool
# - doesn't seem to be a collinearity problem between highschool and pctWhite
# % of commuters who walk to work, % of the population under 15, % of cyclist commuters, 
# and % of males 16-19 unenrolled in school are not significant predictors


                            OLS Regression Results                            
Dep. Variable:                 pctDUI   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     20.29
Date:                Mon, 01 May 2017   Prob (F-statistic):           7.22e-06
Time:                        16:10:31   Log-Likelihood:                -3938.1
No. Observations:                1385   AIC:                             7880.
Df Residuals:                    1383   BIC:                             7891.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      5.2719      0.258     20.399      0.0

In [11]:
print(data_matrix.columns)

yvars = ['pctDUI', 'pctDUI_m1524', 'NumKilled_pctpop', 'NumKilled_pctacc', 'NumAcc_pctpop']

# y values: 
# num killed/ population
# num killed/ num accidents
# num accidents/ pop
# num DUIs / pop: male 15-24 
# num DUIs / num accidents

Index(['GIS_code', 'census_year', 'st_code', 'total', 'urban', 'rural', 'male',
       'female', 'male15_24', 'hh_income', 'drivers', 'solodrivers',
       'carpoolers', 'pubtrans', 'cyclists', 'walkers', 'under15', 'over45',
       'male16_19', 'inschool_m1619', 'unenrolled_m1619', 'unemployed_m1619',
       'lowest_ratio', 'highest_ratio', 'NumKilled', 'NumInjury', 'NumVeh',
       'NumAcc', 'pop', 'medianAge', 'income', 'pctProverty', 'pctWhite',
       'pctBlack', 'pctOccup', 'pctVacant', 'popQT', 'incomeQT', 'provertyQT',
       'occupQT', 'vacantQT', 'hsQT', 'whiteQT', 'blackQT', 'NumKilled_pctpop',
       'NumInjury_pctpop', 'NumVeh_pctpop', 'NumAcc_pctpop',
       'NumKilled_pctacc', 'pctDUI_m1524', 'pctDUI', 'male15_24pct'],
      dtype='object')


In [74]:
#print(X_all.columns)

# matrix of all possible predictor variables:
X_all = data_matrix[['urban', 'male',
        'male15_24', 'hh_income', 'drivers',  
        'carpoolers', 'pubtrans', 'cyclists', 'walkers', 'under15', 'over45',
        'male16_19', 'unenrolled_m1619', 'unemployed_m1619',
        'medianAge','popQT', 'incomeQT', 'provertyQT',
        'vacantQT', 'hsQT', 'whiteQT', 'blackQT']]


X_trimmed = data_matrix[['urban', 'male', 'walkers', 'cyclists','under15', 
                   'unenrolled_m1619', 'carpoolers', 'vacantQT', 'male15_24']]

# look at pairwise correlations between the variables to detect collinearity

np.round(X_trimmed.corr(), 3)

# results are fairly predictable in most cases, surprising in a couple: 
# urban & white are negatively correlated
# poverty & vacancy positively correlated
# vacancy & hh_income negatively, highschool and hh_income positively
# vacancy and highschool negatively correlated (more highschool grads > less vacant)

,urban,male,walkers,cyclists,under15,unenrolled_m1619,carpoolers,vacantQT,male15_24
urban,1.000,-0.238,0.062,0.150,0.103,-0.062,-0.236,-0.329,0.130
male,-0.238,1.000,0.071,0.029,0.030,0.152,0.124,0.107,0.159
walkers,0.062,0.071,1.000,0.329,0.354,0.007,-0.034,0.176,0.342
cyclists,0.150,0.029,0.329,1.000,0.210,-0.015,-0.047,0.067,0.159
under15,0.103,0.030,0.354,0.210,1.000,0.053,0.067,0.114,0.182
unenrolled_m1619,-0.062,0.152,0.007,-0.015,0.053,1.000,0.176,0.225,-0.016
carpoolers,-0.236,0.124,-0.034,-0.047,0.067,0.176,1.000,0.335,-0.036
vacantQT,-0.329,0.107,0.176,0.067,0.114,0.225,0.335,1.000,-0.161
male15_24,0.130,0.159,0.342,0.159,0.182,-0.016,-0.036,-0.161,1.000


In [72]:
X_pctdui = data_matrix[['urban', 'male', 'pubtrans', 'hsQT', 'carpoolers', 'pctWhite', 'male15_24']]
 

np.round(X_pctdui.corr(), 3)

,urban,male,pubtrans,hsQT,carpoolers,pctWhite,male15_24
urban,1.000,-0.238,0.281,0.371,-0.236,-0.412,0.130
male,-0.238,1.000,-0.167,-0.040,0.124,0.248,0.159
pubtrans,0.281,-0.167,1.000,-0.153,-0.005,-0.783,-0.115
hsQT,0.371,-0.040,-0.153,1.000,-0.385,0.129,0.161
carpoolers,-0.236,0.124,-0.005,-0.385,1.000,-0.020,-0.036
pctWhite,-0.412,0.248,-0.783,0.129,-0.020,1.000,0.048
male15_24,0.130,0.159,-0.115,0.161,-0.036,0.048,1.000


In [73]:
linmods = []


y_dui = data_matrix.pctDUI
# y_m1524dui = data_matrix.pctDUI_m1524
# y_pctpop = data_matrix.NumKilled_pctpop
# y_pctacc = data_matrix.NumKilled_pctacc
# let's do

for cat in X_pctdui.columns:
    tempx = X_pctdui[cat]
    tempx = sm.add_constant(tempx)
    templm = sm.OLS(y_dui, tempx).fit()
    linmods.append(templm)
    #print(templm.summary())

# iterate through the list of potential predictors for pctDUI as a dependent var 
# to see individual contribution
# then sorting by p vals

linmods = sorted(linmods, key=lambda x: x.pvalues[1])
print("Percent of accidents which involved a DUI")
# and printing the results
for lm in linmods:
    parameters = list(dict(lm.params).keys())
    print("\n" + parameters[1] + "\n")
    print(lm.summary())

Percent of accidents which involved a DUI

urban

                            OLS Regression Results                            
Dep. Variable:                 pctDUI   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.078
Method:                 Least Squares   F-statistic:                     119.0
Date:                Tue, 02 May 2017   Prob (F-statistic):           1.25e-26
Time:                        10:15:47   Log-Likelihood:                -3894.5
No. Observations:                1386   AIC:                             7793.
Df Residuals:                    1384   BIC:                             7804.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
co

In [90]:
#drugmod = forward_selected(data_matrix[X_pctdui], 'pct_drugs')



In [49]:
# now add the predictors to the model one by one 

# i tested a lot of models individually for pctDUI to get a sense of the stepwise process

#duimod1 = smf.ols(formula = 'pctDUI ~ pctWhite + pubtrans', data=data_matrix).fit()
# ^^ pubtrans is insignificant
#duimod1 =  smf.ols(formula = 'pctDUI ~ hsQT + pubtrans', data=data_matrix).fit()
# ^^ not bad, p-val: 5*10^-19
duimod1 =  smf.ols(formula = 'pctDUI ~ urban + pubtrans + hsQT', data=data_matrix).fit()

# ^^ r-squared: 0.095, p-valL 10^-28 - and coeff is more negative for pubtrans, largest for hsQT
# ^ BEST SO FAR
# duimod2 =  smf.ols(formula = 'pctDUI ~ urban + pubtrans', data=data_matrix).fit()
# ^^ r-squared: 0.092, p-val:10^-28 - interestingly, urban and pubtrans are not highly collinear

#duimod3 =  smf.ols(formula = 'pctDUI ~ pubtrans + carpoolers + urban + hsQT', data=data_matrix).fit()
#duimod4 =  smf.ols(formula = 'pctDUI ~ pubtrans + carpoolers', data=data_matrix).fit()
# ^^ carpoolers is not a significant predictor when added to the model

# duimod3 =  smf.ols(formula = 'pctDUI ~ urban + hsQT', data=data_matrix).fit()  
# ^^ hsQT entirely insignificant in this model
# duimod3 = smf.ols(formula = 'pctDUI ~ hsQT', data=data_matrix).fit()
# ^^ hsQT is significant but R-squared is teeny - 0.014 
# duimod3 = smf.ols(formula = 'pctDUI ~ pctWhite + hsQT', data=data_matrix).fit()
# ^^ hsQT much more signif in this model than the one below (smaller p-val, larger coeff), 
# but including urban still results in all three predictors being significant and a higher R-squared value

# duimod4 =  smf.ols(formula = 'pctDUI ~ male + pubtrans', data=data_matrix).fit()
# duimod4 =  smf.ols(formula = 'pctDUI ~ male + pctWhite', data=data_matrix).fit()
# duimod5 =  smf.ols(formula = 'pctDUI ~ urban + pubtrans + male', data=data_matrix).fit()
# duimod4 =  smf.ols(formula = 'pctDUI ~ urban + male', data=data_matrix).fit()
# ^^ male is insignificant in all four models; gonna stop considering it 

#duimod5 = smf.ols(formula = 'pctDUI ~ urban + pctWhite + hsQT + pubtrans', data=data_matrix).fit()
# ^^ pctWhite becomes insignificant in this model

#duimods = [duimod1, duimod2, duimod3, duimod4]
#for d in duimods:
 #   print(d.summary())

# BEST MODEL:
print(duimod1.summary())




                            OLS Regression Results                            
Dep. Variable:                 pctDUI   R-squared:                       0.095
Model:                            OLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     48.24
Date:                Tue, 02 May 2017   Prob (F-statistic):           1.20e-29
Time:                        09:51:36   Log-Likelihood:                -3879.2
No. Observations:                1385   AIC:                             7766.
Df Residuals:                    1381   BIC:                             7787.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      6.5299      0.273     23.947      0.0

In [92]:
# now trying drugs&alc involved accidents

X_withY = list(X_pctdui.columns)

X_withY.append('pct_drugsalc')

mod_pctDUI = forward_selected(data_matrix[X_withY], 'pct_drugsalc')

#print(X_withY)
print(mod_pctDUI.model.formula)

print(mod_pctDUI.model.fit().summary())

pct_drugsalc ~ male15_24 + pctWhite + urban + male + 1
                            OLS Regression Results                            
Dep. Variable:           pct_drugsalc   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     2.289
Date:                Tue, 02 May 2017   Prob (F-statistic):             0.0610
Time:                        11:54:24   Log-Likelihood:                -399.90
No. Observations:                 217   AIC:                             809.8
Df Residuals:                     212   BIC:                             826.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
----------------------------------------------------------------------------

In [17]:
xx = list(X_all.columns)
xx.append('NumKilled_pctacc')
X_withY
# use forward_select function to determine most signif variables
mod_pctacc = forward_selected(data_matrix[xx], 'NumKilled_pctacc')
print(mod_pctacc.model.formula)
#print(mod_pctacc.model.fit().summary())

NumKilled_pctacc ~ urban + male + male15_24 + walkers + drivers + pubtrans + incomeQT + cyclists + pop + unenrolled_m1619 + blackQT + over45 + 1


In [35]:
# the most influential variables, with highly correlated vars removed

X_pctacc = data_matrix[['urban', 'male', 'male15_24', 'walkers', 'pubtrans', 'incomeQT', 'cyclists', 
                        'unenrolled_m1619']]


np.round(X_pctacc.corr(), 3)

,urban,male,male15_24,walkers,pubtrans,incomeQT,cyclists,unenrolled_m1619
urban,1.000,-0.276,0.128,0.061,0.281,0.270,0.150,-0.062
male,-0.276,1.000,0.157,0.070,-0.185,-0.009,0.027,0.166
male15_24,0.128,0.157,1.000,0.341,-0.116,-0.055,0.159,-0.015
walkers,0.061,0.070,0.341,1.000,0.098,-0.063,0.329,0.007
pubtrans,0.281,-0.185,-0.116,0.098,1.000,-0.113,0.081,0.054
incomeQT,0.270,-0.009,-0.055,-0.063,-0.113,1.000,0.022,-0.223
cyclists,0.150,0.027,0.159,0.329,0.081,0.022,1.000,-0.015
unenrolled_m1619,-0.062,0.166,-0.015,0.007,0.054,-0.223,-0.015,1.000


In [75]:
# Number of Fatal Accidents by Number of Accidents Total: 

#pctacc1 =  smf.ols(formula = 'NumKilled_pctacc ~ urban + male', data=data_matrix).fit()

# BEST MODEL: (pctacc2)
pctacc1 = smf.ols(formula = 'NumKilled_pctacc ~ urban + male + walkers + unenrolled_m1619', data=data_matrix).fit()
pctacc2 = smf.ols(formula = 'NumKilled_pctacc ~ urban + male + male15_24pct + walkers + unenrolled_m1619', data=data_matrix).fit()


#pctacc4 = smf.ols(formula = 'NumKilled_pctacc ~ urban + male + male15_24 + walkers + unenrolled_m1619 + pubtrans + cyclists', data=data_matrix).fit()

# over45, incomeQT not signif once added to the model 

pctacc = [pctacc1, pctacc2]

for p in pctacc:
    print(p.summary())



                            OLS Regression Results                            
Dep. Variable:       NumKilled_pctacc   R-squared:                       0.140
Model:                            OLS   Adj. R-squared:                  0.137
Method:                 Least Squares   F-statistic:                     56.01
Date:                Tue, 02 May 2017   Prob (F-statistic):           7.97e-44
Time:                        10:24:55   Log-Likelihood:                -6163.5
No. Observations:                1386   AIC:                         1.234e+04
Df Residuals:                    1381   BIC:                         1.236e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------
Intercept          -30.2828      8.404  

In [76]:
LMs = []

X_pctpop = data_matrix[['urban', 'popQT', 'blackQT', 'provertyQT', 'male', 'cyclists', 'male15_24pct',
                        'hh_income', 'under15']]


y_pctpop = data_matrix.NumKilled_pctpop
# let's do

for cat in X_pctpop.columns:
    tempx = X_pctpop[cat]
    tempx = sm.add_constant(tempx)
    templm = sm.OLS(y_pctpop, tempx).fit()
    LMs.append(templm)
    #print(templm.summary())

# iterate through the list of potential predictors for pctDUI as a dependent var 
# to see individual contribution
# then sorting by p vals

LMs = sorted(LMs, key=lambda x: x.pvalues[1])
print("Number of fatalities by population")
# and printing the results
for lm in LMs:
    parameters = list(dict(lm.params).keys())
    print("\n" + parameters[1] + "\n")
    print(lm.summary())

Number of fatalities by population

urban

                            OLS Regression Results                            
Dep. Variable:       NumKilled_pctpop   R-squared:                       0.087
Model:                            OLS   Adj. R-squared:                  0.086
Method:                 Least Squares   F-statistic:                     131.3
Date:                Tue, 02 May 2017   Prob (F-statistic):           4.10e-29
Time:                        10:25:29   Log-Likelihood:                -3119.7
No. Observations:                1386   AIC:                             6243.
Df Residuals:                    1384   BIC:                             6254.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const    

In [61]:
# Number of people killed by census tract population

pctpop1 = smf.ols(formula = 'NumKilled_pctpop ~ urban + male + hh_income + popQT + under15', data=data_matrix).fit()
#pctpop2 = smf.ols(formula = 'NumKilled_pctpop ~ urban + male + hh_income + popQT + under15', data=data_matrix).fit()
#pctpop3 = smf.ols(formula = 'NumKilled_pctpop ~ urban + male + hh_income + popQT + under15 + cyclists', data=data_matrix).fit()

print(pctpop1.summary())
#pctacc4 = smf.ols(formula = 'NumKilled_pctacc ~ urban + male + male15_24 + walkers + unenrolled_m1619 + pubtrans + cyclists', data=data_matrix).fit()

# over45, incomeQT not signif once added to the model 

# pctpop = [pctpop1, pctpop2, pctpop3]

# for p in pctpop:
#     print(p.summary())

                            OLS Regression Results                            
Dep. Variable:       NumKilled_pctpop   R-squared:                       0.106
Model:                            OLS   Adj. R-squared:                  0.102
Method:                 Least Squares   F-statistic:                     32.61
Date:                Tue, 02 May 2017   Prob (F-statistic):           1.63e-31
Time:                        10:10:15   Log-Likelihood:                -3103.2
No. Observations:                1385   AIC:                             6218.
Df Residuals:                    1379   BIC:                             6250.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      1.4659      1.018      1.439      0.1